# Noise Meter Demo 💥🔈

### This code should:
- Turn button yellow & listen for when sounds exceed 0.3 RMS
- When you want to hear the sounds that have been recorded, STOP THE CELL and run the next one.
- The button will turn purple. When you press it, it will list out the names of the files that were recorded that day.
- Then the button will turn blue. This will play all the sounds with a two second wait in between.
- All sound files are in the navigator in .wav format - you can export them if needed.

### Notes:

The third cell will give LOTS of error messages. Ignore them.

Make sure to run the last cell before re-running the cell that records you, or you'll get errors.

In [1]:
import pyaudio
import time
from math import log10
import audioop

from IPython.display import clear_output
from aiy.voice.audio import AudioFormat, play_wav, record_file

import glob
from aiy.pins import BUTTON_GPIO_PIN
from gpiozero import Button
from aiy.leds import Leds, Color

buttonLight = Leds()

In [2]:
p = pyaudio.PyAudio()

WIDTH = 2
RATE = int(p.get_default_input_device_info()['defaultSampleRate'])
DEVICE = p.get_default_input_device_info()['index']
rms = 1

ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.front
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround40
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround41
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround50
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround51
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround71
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2565:(snd

In [3]:
print(p.get_default_input_device_info())

{'index': 7, 'structVersion': 2, 'name': 'default', 'hostApi': 0, 'maxInputChannels': 128, 'maxOutputChannels': 128, 'defaultLowInputLatency': 0.005804988662131519, 'defaultLowOutputLatency': 0.008707482993197279, 'defaultHighInputLatency': 0.034829931972789115, 'defaultHighOutputLatency': 0.034829931972789115, 'defaultSampleRate': 44100.0}


In [4]:
def callback(in_data, frame_count, time_info, status):
    global rms
    rms = audioop.rms(in_data, WIDTH) / 32767
    return in_data, pyaudio.paContinue

stream = p.open(format=p.get_format_from_width(WIDTH),
                input_device_index=DEVICE,
                channels=1,
                rate=RATE,
                input=True,
                output=False,
                stream_callback=callback)

In [5]:
import time

recdur = 5

stream.start_stream()

buttonLight.update(buttonLight.rgb_on((255, 100, 0)))

try:
    while stream.is_active(): 
        # print out the RMS and DB values
        db = 20 * log10(rms)
        print(f"RMS: {rms:.4f} DB: {db:.1f}") 
        clear_output(wait=True)
        
        if (rms > 0.3):
            # stop stream
            stream.stop_stream()
            stream.close()
            
            # angrily record loud people 
            print("You're being too loud 😡 I will record you!")
            
            date = time.strftime("%Y%m%d-%H%M%S")
            fn = "loud-" + date + ".wav"
            record_file(AudioFormat.CD, filename=fn, filetype='wav',
                wait=lambda: time.sleep(recdur))
            
            # start stream
            stream = p.open(format=p.get_format_from_width(WIDTH),
                input_device_index=DEVICE,
                channels=1,
                rate=RATE,
                input=True,
                output=False,
                stream_callback=callback)
            stream.start_stream()
            
        
        # refresh every 0.3 seconds 
        time.sleep(0.3)

except KeyboardInterrupt:
    print('Done')

stream.stop_stream()
stream.close()

Done


In [6]:
today = time.strftime("%Y%m%d")

global printedSnds 
printedSnds = False

def playSounds():
    print("Pressed")
    global printedSnds
    files = glob.glob("loud-" + today + "*.wav")
    for file in files:
        if (not printedSnds):
            print("Printin' it")
            print(file)
        else:
            print("Playin' it")
            play_wav(file)
            sleep(2)
    printedSnds = True
        
        
with Leds() as buttonLight, Button(BUTTON_GPIO_PIN) as button:
    if (printedSnds):
        buttonLight.update(buttonLight.rgb_on((0, 0, 255)))
    else:
        buttonLight.update(buttonLight.rgb_on((255, 0, 255)))
    while True:
        button.when_pressed=playSounds

KeyboardInterrupt: 

In [ ]:
stream.close()
p.terminate()